In [2]:
import torch
import numpy as np
from tqdm.auto import tqdm
import cloudpickle as pickle
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

In [2]:
n = 10
x = torch.tensor(np.random.randn(2, n), dtype=torch.float32, requires_grad=True)
y = torch.tensor(np.random.randn(2, n) * 10, dtype=torch.float32, requires_grad = False)
bn = torch.nn.BatchNorm1d(num_features=n, affine=True)

In [8]:
mse = torch.nn.MSELoss()
params = dict(bn.named_parameters())
all_ones = torch.tensor(np.ones(n), dtype=torch.float32, requires_grad=False)
all_zeros = torch.tensor(np.zeros(n), dtype=torch.float32, requires_grad=False)
regul_loss = mse(params['weight'], all_ones) + mse(params['bias'], all_zeros)

In [9]:
opt = torch.optim.Adam(bn.parameters())

In [20]:
for _ in tqdm(range(1000)):
    opt.zero_grad()
    regul_loss = mse(params['weight'], all_ones) + mse(params['bias'], all_zeros)
    fit_loss = mse(bn(x), y)
    loss = fit_loss + regul_loss
    loss.backward()
    opt.step()
fit_loss, regul_loss

(tensor(35.3417, grad_fn=<MseLossBackward>),
 tensor(17.7487, grad_fn=<AddBackward0>))

In [22]:
params

{'weight': Parameter containing:
 tensor([ 1.5248, -0.5651, -0.7201, -3.3905,  3.2868, -2.6270, -3.6368,  2.8743,
         -3.5234, -1.2139], requires_grad=True),
 'bias': Parameter containing:
 tensor([-1.1832,  1.3406,  3.8134, -4.8872, -2.1060, -0.5980, -0.3938,  3.0472,
          3.9349,  3.5296], requires_grad=True)}

In [24]:
dir(bn)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__constants__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_check_input_dim',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_forward_unimplemented',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_module',
 'affine',
 'apply',
 'bfloat16',


In [7]:
fn = '/home/sergei/ray_results/ve5_vf_linreg_poc/main_fcn_1f3e8_00000_0_2020-12-07_16-01-45/checkpoint_1000/checkpoint'

In [8]:
learner = pickle.load(open(fn, 'rb'))

Make environment VectorIncrement-v0 None {}
Using device cpu


In [9]:
list(learner.decoder.bn.named_parameters())

[('weight',
  Parameter containing:
  tensor([1.0261, 1.0600, 1.0447, 1.0430, 1.0474], requires_grad=True)),
 ('bias',
  Parameter containing:
  tensor([-0.0766,  0.0223, -0.0542,  0.0556,  0.0548], requires_grad=True))]